In [12]:
import pandas as pd
import featuretools as ft
from woodwork.logical_types import Categorical, Double, Datetime, Age

In [10]:
path = '../data/car_train.csv'
car_info = pd.read_csv(path) 

path = '../data/rides_info.csv'  
rides_info = pd.read_csv(path)

path = '../data/driver_info.csv' 
driver_info = pd.read_csv(path) 

path = '../data/fix_info.csv'
fix_info = pd.read_csv(path)

In [11]:
es = ft.EntitySet(id='car_data')
ft.list_logical_types()

,name,type_string,description,physical_type,standard_tags,is_default_type,is_registered,parent_type
0,Address,address,Represents Logical Types that contain address ...,string,{},True,True,None
1,Age,age,Represents Logical Types that contain whole nu...,int64,{numeric},True,True,Integer
2,AgeFractional,age_fractional,Represents Logical Types that contain non-nega...,float64,{numeric},True,True,Double
3,AgeNullable,age_nullable,Represents Logical Types that contain whole nu...,Int64,{numeric},True,True,IntegerNullable
4,Boolean,boolean,Represents Logical Types that contain binary v...,bool,{},True,True,BooleanNullable
5,BooleanNullable,boolean_nullable,Represents Logical Types that contain binary v...,boolean,{},True,True,None
6,Categorical,categorical,Represents Logical Types that contain unordere...,category,{category},True,True,None
7,CountryCode,country_code,Represents Logical Types that use the ISO-3166...,category,{category},True,True,Categorical
8,CurrencyCode,currency_code,Represents Logical Types that use the ISO-4217...,category,{category},True,True,Categorical
9,Datetime,datetime,Represents Logical Types that contain date and...,datetime64[ns],{},True,True,None


In [17]:
es = es.add_dataframe(
    dataframe_name="cars",
    dataframe=car_info,
    index="car_id",
    logical_types={"car_type": Categorical, 'fuel_type': Categorical, 'model': Categorical}
    )

es = es.add_dataframe(
    dataframe_name="rides",
    dataframe=rides_info.drop(['ride_id'], axis=1),
    index='index',
    time_index="ride_date",
    )

es = es.add_dataframe(
    dataframe_name="drivers",
    dataframe=driver_info,
    index="user_id",
    logical_types={"sex": Categorical, "first_ride_date": Datetime, "age": Age}
    )

es = es.add_dataframe(
    dataframe_name="fixes",
    dataframe=fix_info,
    index="index",
    logical_types={"work_type": Categorical, "worker_id":Categorical}
    )
es

/home/hallteon/.local/lib/python3.8/site-packages/featuretools/entityset/entityset.py:1910: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/home/hallteon/.local/lib/python3.8/site-packages/woodwork/type_sys/utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pd.to_datetime(
/home/hallteon/.local/lib/python3.8/site-packages/woodwork/type_sys/utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pd.to_datetime(
/home/hallteon/.local/lib/python3.8/site-packages/woodwork/type_sys/utils.py:40: UserWarni

Entityset: car_data
  DataFrames:
    cars [Rows: 2337, Columns: 10]
    rides [Rows: 739500, Columns: 14]
    drivers [Rows: 15153, Columns: 7]
    fixes [Rows: 146000, Columns: 7]
  Relationships:
    No relationships

In [19]:
es = es.add_relationship("cars", "car_id", "rides", "car_id")
es = es.add_relationship("drivers", "user_id", "rides", "user_id")
es = es.add_relationship("cars", "car_id", "fixes", "car_id")

es

/home/hallteon/.local/lib/python3.8/site-packages/featuretools/entityset/entityset.py:379: UserWarning: Not adding duplicate relationship: <Relationship: rides.car_id -> cars.car_id>
  warnings.warn("Not adding duplicate relationship: " + str(relationship))
/home/hallteon/.local/lib/python3.8/site-packages/featuretools/entityset/entityset.py:379: UserWarning: Not adding duplicate relationship: <Relationship: rides.user_id -> drivers.user_id>
  warnings.warn("Not adding duplicate relationship: " + str(relationship))
/home/hallteon/.local/lib/python3.8/site-packages/featuretools/entityset/entityset.py:379: UserWarning: Not adding duplicate relationship: <Relationship: fixes.car_id -> cars.car_id>
  warnings.warn("Not adding duplicate relationship: " + str(relationship))


Entityset: car_data
  DataFrames:
    cars [Rows: 2337, Columns: 10]
    rides [Rows: 739500, Columns: 14]
    drivers [Rows: 15153, Columns: 7]
    fixes [Rows: 146000, Columns: 7]
  Relationships:
    rides.car_id -> cars.car_id
    rides.user_id -> drivers.user_id
    fixes.car_id -> cars.car_id

In [31]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='cars',
    agg_primitives=['sum', 'count', 'max'],
    instance_ids=car_info.loc[car_info['model'] == 'Tesla Model 3'].index,
    max_depth=1,
)

feature_matrix

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,COUNT(rides),...,SUM(rides.ride_duration),SUM(rides.speed_avg),SUM(rides.speed_max),SUM(rides.stop_times),SUM(rides.user_ride_quality),COUNT(fixes),MAX(fixes.destroy_degree),MAX(fixes.work_duration),SUM(fixes.destroy_degree),SUM(fixes.work_duration)
car_id,,,,,,,,,,,,,,,,,,,,,
O-2124190y,Tesla Model 3,premium,electro,3.90,2017,116872,2019,50.26,gear_stick,174,...,215095.0,7630.0,11601.438521,147.0,-707.814812,35,10.0,61.0,113.1,970.0
n19871201N,Tesla Model 3,premium,electro,3.58,2016,89811,2018,100.42,another_bug,174,...,236549.0,7786.0,16070.296888,694.0,1127.610285,35,10.0,68.0,124.6,858.0
I13914989Z,Tesla Model 3,premium,electro,4.64,2016,98304,2019,100.62,engine_ignition,174,...,290518.0,8935.0,16999.476225,139.0,1929.941778,35,5.0,65.0,115.0,896.0
g-2283000q,Tesla Model 3,premium,electro,5.44,2015,75113,2020,47.95,gear_stick,174,...,319610.0,7845.0,11860.793108,131.0,-380.108901,35,9.3,60.0,119.0,796.0
i-1106718P,Tesla Model 3,premium,electro,4.72,2015,75786,2017,119.84,engine_ignition,174,...,330058.0,8978.0,16625.210818,134.0,-2640.911012,35,5.0,65.0,92.4,914.0
I-2073229s,Tesla Model 3,premium,electro,5.10,2017,120231,2017,59.54,engine_fuel,174,...,315890.0,8847.0,15495.264851,137.0,-1315.978911,35,10.0,61.0,129.1,890.0
N44849090k,Tesla Model 3,premium,electro,3.60,2015,86115,2022,55.30,electro_bug,174,...,309597.0,8699.0,15936.226133,159.0,450.513087,34,9.0,48.0,111.5,864.0
T20813001U,Tesla Model 3,premium,electro,3.98,2017,132784,2017,55.18,engine_fuel,174,...,288346.0,8507.0,14530.796406,127.0,-1773.555746,34,10.0,67.0,133.9,848.0
O-2688441F,Tesla Model 3,premium,electro,4.82,2015,80038,2021,56.28,engine_fuel,174,...,192238.0,8766.0,14688.656868,145.0,686.630703,34,10.0,71.0,114.3,914.0


In [33]:
feature_matrix.columns

Index(['model', 'car_type', 'fuel_type', 'car_rating', 'year_to_start',
       'riders', 'year_to_work', 'target_reg', 'target_class', 'COUNT(rides)',
       'MAX(rides.deviation_normal)', 'MAX(rides.distance)',
       'MAX(rides.rating)', 'MAX(rides.refueling)', 'MAX(rides.ride_cost)',
       'MAX(rides.ride_duration)', 'MAX(rides.speed_avg)',
       'MAX(rides.speed_max)', 'MAX(rides.stop_times)',
       'MAX(rides.user_ride_quality)', 'SUM(rides.deviation_normal)',
       'SUM(rides.distance)', 'SUM(rides.rating)', 'SUM(rides.refueling)',
       'SUM(rides.ride_cost)', 'SUM(rides.ride_duration)',
       'SUM(rides.speed_avg)', 'SUM(rides.speed_max)', 'SUM(rides.stop_times)',
       'SUM(rides.user_ride_quality)', 'COUNT(fixes)',
       'MAX(fixes.destroy_degree)', 'MAX(fixes.work_duration)',
       'SUM(fixes.destroy_degree)', 'SUM(fixes.work_duration)'],
      dtype='object')

In [34]:
feature_matrix['MAX(rides.speed_max)'].max()

199.3701025284091

In [35]:
feature_matrix['COUNT(fixes)'].mean()

34.42857142857143

In [36]:
feature_matrix['SUM(fixes.work_duration)'].max()

970.0

In [23]:
car_info['model'].value_counts()

model
Hyundai Solaris       161
Smart ForFour         158
Renault Kaptur        154
Renault Sandero       152
VW Polo VI            152
Kia Sportage          147
Skoda Rapid           147
Nissan Qashqai        146
Kia Rio X             143
VW Polo               141
Kia Rio X-line        140
VW Tiguan             135
Smart ForTwo          130
Smart Coupe           118
Kia Rio               111
Mercedes-Benz E200     23
Mini Cooper            22
Audi Q3                21
MINI CooperSE          21
Audi A4                20
Fiat 500               18
Audi A3                17
Mercedes-Benz GLC      17
BMW 320i               16
Tesla Model 3          14
Volkswagen ID.4        13
Name: count, dtype: int64

In [2]:
!pip install featuretools -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
